In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import pickle
import matplotlib.pyplot as plt
import ast
import json

In [35]:
edges = pd.read_csv('../assets/edges.csv')
nodes = pd.read_csv('../assets/nodes.csv')

In [36]:
edges

,id_0,id_1
0,76M2Ekj8bG8W7X2nbx2CpF,7sfl4Xt5KmfyDs2T3SVSMK
1,0hk4xVujcyOr6USD95wcWb,7Do8se3ZoaVqUt3woqqSrD
2,38jpuy3yt3QIxQ8Fn1HTeJ,4csQIMQm6vI2A2SCVDuM2z
3,6PvcxssrQ0QaJVaBWHD07l,6UCQYrcJ6wab6gnQ89OJFh
4,2R1QrQqWuw3IjoP5dXRFjt,4mk1ScvOUkuQzzCZpT6bc0
...,...,...
300381,0PK0Dx3s9et0Uf4XbdFpiW,6SmpksRq3jxeDZ7roKDb6F
300382,5hqRsNHDZH1jHzI9LgxFRZ,7KYseTkErcYMRvcIqacxUq
300383,0wjb0t7aYMuIwyuXnPOilh,2p1fiYHYiXz9qi0JJyxBzN
300384,3ciRvbBIVz9fBoPbtSYq4x,5einkgXXrjhfYCyac1FANB


In [37]:
nodes = nodes.drop("chart_hits", axis=1)
nodes['genres'] = nodes['genres'].apply(ast.literal_eval)
nodes = nodes[nodes['genres'].apply(lambda x: len(x) > 0)]
popularity = nodes["popularity"].to_list()

In [38]:
# from collections import Counter
# c = Counter(popularity)
# c = sorted(c.items(), key= lambda x:-x[0])

# brk = 15000
# ct = 0
# e = 0
# for (k, v) in c:
#     ct += v
#     if ct >= brk:
#         e = k
#         print(k)
#         break

In [39]:
# nodes = nodes.loc[nodes['popularity'] >= e]
nodes = nodes[~nodes['name'].duplicated(keep=False)]
# nodes

In [40]:
nodes

,spotify_id,name,followers,popularity,genres
0,48WvrUGoijadXXCsGocwM4,Byklubben,1738.0,24,"[nordic house, russelater]"
1,4lDiJcOJ2GLCK6p9q5BgfK,Kontra K,1999676.0,72,"[christlicher rap, german hip hop]"
3,3dXC1YPbnQPsfHPVkm1ipj,Christopher Martin,249233.0,52,"[dancehall, lovers rock, modern reggae, reggae..."
4,74terC9ol9zMo8rfzhSOiG,Jakob Hellman,21193.0,39,"[classic swedish pop, norrbotten indie, swedis..."
7,3TG1RXLaEhHz5SIPMWahit,Nehuda,36252.0,31,[francoton]
...,...,...,...,...,...
156392,6HRnGwZ87Xob0I0cHSvr0s,Sevda Alekperzadeh,5634.0,8,[azeri pop]
156396,0By75sLZQ06mWyzNiSdLUB,Menor,26346.0,39,[funk carioca]
156402,7vlW9kOjO3ioYJ0k0P6xCG,Grupo Los Generales,2226.0,33,[corrido]
156413,6mrB3jdXxZq1dl8m8eOVFh,Getamilli,337.0,16,[ritmo kombina]


In [41]:
genres = nodes["genres"].to_list()
all_genres = set()
for g in genres:
    l = set(g)
    all_genres = all_genres.union(l)

In [42]:
columns = ["id", "name", "followers", "popularity"] + list(all_genres)
genres_to_ind = {}

for i in range(len(columns)):
    genres_to_ind[columns[i]] = i

In [43]:
data = []

for ind, row in nodes.iterrows():
    r = [0] * len(columns)
    r[0] = (row["spotify_id"])
    r[1] = (row["name"])
    r[2] = (row["followers"])
    r[3] = (row["popularity"])
    for g in row["genres"]:
        r[genres_to_ind[g]] = 1

    data.append(r)

df = pd.DataFrame(data, columns=columns)
df = df.loc[:, df.sum() != 0]

In [44]:
ind = enumerate(df.sum().to_list()[4:])
sorted_inds = sorted(ind, key= lambda x:-x[1])

In [45]:
cols = df.columns

for (u, v) in sorted_inds:
    print(cols[u + 4], v)

sel_cols = [cols[u + 4] for (u, v) in sorted_inds[0:15]]

pop 540
electro house 508
dance pop 492
edm 449
rap 444
german hip hop 410
uk pop 400
pop rap 391
hip hop 376
pop dance 373
french hip hop 366
pop edm 364
tropical house 348
filmi 347
electropop 343
funk carioca 342
trap 339
desi pop 325
pop urbaine 306
trap latino 296
r&b 294
dutch hip hop 291
house 279
electronica 276
rock 276
k-pop 273
gangster rap 271
latin pop 271
underground hip hop 271
southern hip hop 268
mandopop 265
polish hip hop 262
modern bollywood 253
rap francais 253
swedish pop 252
reggaeton 251
urban contemporary 246
classical performance 246
taiwan pop 246
reggaeton flow 243
funk mtg 242
nueva musica mexicana 242
russian hip hop 241
progressive house 240
norwegian pop 237
j-pop 234
latin hip hop 234
indie poptimism 232
tropical 231
musica mexicana 231
dutch pop 229
v-pop 227
modern rock 226
alternative r&b 224
italian hip hop 224
danish pop 223
finnish dance pop 221
eurovision 219
post-teen pop 217
pop rock 217
francoton 217
german pop 216
uk hip hop 210
finnish hip h

In [46]:
sel_cols

['pop',
 'electro house',
 'dance pop',
 'edm',
 'rap',
 'german hip hop',
 'uk pop',
 'pop rap',
 'hip hop',
 'pop dance',
 'french hip hop',
 'pop edm',
 'tropical house',
 'filmi',
 'electropop']

In [47]:
df_sel = df[sel_cols]

In [48]:
df_sel

,pop,electro house,dance pop,edm,rap,german hip hop,uk pop,pop rap,hip hop,pop dance,french hip hop,pop edm,tropical house,filmi,electropop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52858,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52859,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
df_non_zero = df_sel.loc[df_sel.sum(axis=1) == 1]

In [50]:
df_non_zero_res = df_non_zero.sum().to_dict()

In [51]:
df_non_zero_res

{'pop': 155,
 'electro house': 194,
 'dance pop': 103,
 'edm': 34,
 'rap': 90,
 'german hip hop': 410,
 'uk pop': 181,
 'pop rap': 143,
 'hip hop': 106,
 'pop dance': 27,
 'french hip hop': 366,
 'pop edm': 192,
 'tropical house': 129,
 'filmi': 347,
 'electropop': 141}

In [52]:
cols = ['pop', 'electro house', 'german hip hop', 'uk pop', 'hip hop', 'filmi']

In [102]:
df_final = df[cols]

In [103]:
df_final_non_zero = df_final.loc[df_final.sum(axis=1) == 1]

In [104]:
def get_non_zero_column(row):
    for col in row.index:
        if row[col] != 0:
            return col

In [106]:
df_final_non_zero.loc[:, 'NonZero'] = df_final_non_zero.apply(get_non_zero_column, axis=1)

In [108]:
df_final_non_zero.loc[:, 'NonZero_Number'] = pd.Categorical(df_final_non_zero['NonZero']).codes

In [110]:
factor_map = {code: category for code, category in enumerate(pd.Categorical(df_final_non_zero["NonZero"]).categories)}

In [111]:
factor_map

{0: 'electro house',
 1: 'filmi',
 2: 'german hip hop',
 3: 'hip hop',
 4: 'pop',
 5: 'uk pop'}

In [112]:
with open("../components/classification/categories.json", "w") as file:
    json.dump(factor_map, file, indent=2)

In [113]:
df_final_non_zero

,pop,electro house,german hip hop,uk pop,hip hop,filmi,NonZero,NonZero_Number
1,0,0,1,0,0,0,german hip hop,2
23,0,0,0,0,1,0,hip hop,3
25,0,0,0,1,0,0,uk pop,5
27,0,0,1,0,0,0,german hip hop,2
28,0,1,0,0,0,0,electro house,0
...,...,...,...,...,...,...,...,...
52465,0,0,0,0,0,1,filmi,1
52709,0,0,0,0,1,0,hip hop,3
52717,0,1,0,0,0,0,electro house,0
52753,0,1,0,0,0,0,electro house,0


In [114]:
final_inds = df_final_non_zero.index

In [115]:
final_inds

Index([    1,    23,    25,    27,    28,    33,    60,    85,   102,   142,
       ...
       52341, 52371, 52376, 52442, 52459, 52465, 52709, 52717, 52753, 52776],
      dtype='int64', length=2196)

In [118]:
df_only_selected = df.loc[final_inds]

In [121]:
df_only_selected.loc[:, 'label'] = df_final_non_zero['NonZero_Number']

In [122]:
df_only_selected

,id,name,followers,popularity,pop paraguayo,folkmusik,comedy rap,czsk reggae,birmingham indie,rock nica,...,mindfulness,nice indie,tololoche,rochester ny indie,schweizer rap,classic j-rock,latin hip hop,cape verdean folk,thai hip hop,label
1,4lDiJcOJ2GLCK6p9q5BgfK,Kontra K,1999676.0,72,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
23,5Matrg5du62bXwer29cU5T,Benny The Butcher,266547.0,64,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
25,4e0nWw2r4BoQSKPQ2zpU13,Ray Dalton,39139.0,71,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
27,0H1pP8C4zaVSilRjiRa2Wy,D1NO,19269.0,48,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
28,2HmmA3GhP0xsbRUiDllJUn,Pep & Rash,77599.0,39,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52465,2dHiQ5ggS4E4VguS5f9Ap4,Rimi Tomy,300705.0,45,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
52709,3BeQqzKdlARoOd6y30kCO2,Pete Rock,298190.0,53,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
52717,3a9qv6NLHnsVxJUtKOMHvD,The Glitch Mob,538974.0,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52753,6JtWu7EQUhFCLNZU92kq1o,Schoolboy,10153.0,25,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
df_only_selected_final = df_only_selected[['id', 'name', 'followers', 'popularity', 'label']].reset_index().drop('index', axis=1)

In [126]:
df_only_selected_final

,id,name,followers,popularity,label
0,4lDiJcOJ2GLCK6p9q5BgfK,Kontra K,1999676.0,72,2
1,5Matrg5du62bXwer29cU5T,Benny The Butcher,266547.0,64,3
2,4e0nWw2r4BoQSKPQ2zpU13,Ray Dalton,39139.0,71,5
3,0H1pP8C4zaVSilRjiRa2Wy,D1NO,19269.0,48,2
4,2HmmA3GhP0xsbRUiDllJUn,Pep & Rash,77599.0,39,0
...,...,...,...,...,...
2191,2dHiQ5ggS4E4VguS5f9Ap4,Rimi Tomy,300705.0,45,1
2192,3BeQqzKdlARoOd6y30kCO2,Pete Rock,298190.0,53,3
2193,3a9qv6NLHnsVxJUtKOMHvD,The Glitch Mob,538974.0,61,0
2194,6JtWu7EQUhFCLNZU92kq1o,Schoolboy,10153.0,25,0


In [127]:
df_only_selected_final.to_csv('../components/classification/master.csv')

In [129]:
id_map = pd.Series(df_only_selected_final.index, index=df_only_selected_final["id"]).to_dict()
name_map = pd.Series(df_only_selected_final.index, index=df_only_selected_final["name"]).to_dict()

In [130]:
ind_to_name = {v:k for (k, v) in name_map.items()}

### graph

In [131]:
n = len(name_map)
A = np.zeros((n, n))

ids = set(id_map.keys())

for _, row in edges.iterrows():
    u, v = row["id_0"], row["id_1"]

    if u in ids and v in ids and u != v:
        ui, vi = id_map[u], id_map[v]
        A[ui][vi] = 1
        A[vi][ui] = 1

In [132]:
G = nx.from_numpy_array(A)

In [133]:
ind_to_name = {v:k for (k, v) in name_map.items()}

In [134]:
empty = []
for node in G.nodes():
    if G.degree[node] == 0:
        empty.append(node)

ccs = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]
for cc in ccs[1:]:
    pcc = []
    for a in cc:
        pcc.append(ind_to_name[a])
        empty.append(a)
    print(pcc)

['JUNNY', 'Gentle Bones', 'lullaboy', 'Rangga Jones']
['Cate', 'Cian Ducrot']
['Finding Hope', 'Direct']
['Matt Terry', 'Samantha Harvey']
['VINCINT', 'Qveen Herby']
['Rachel Grae', 'Boyce Avenue']
['VAX', 'Ellise']
['mxmtoon', 'Ricky Montgomery']
['D1NO']
['Samuel Jack']
['Amanda Seyfried']
['Manu Gavassi']
['Zensery']
['Jamie Lawson']
['Luke Chiang']
['MC Bilal']
['Matt Maltese']
['Maddie & Tae']
['Tom Speight']
['Folly Rae']
['brb.']
['One Direction']
['N.W.A.']
['CIVO']
['Life of Dillon']
['ASTN']
['Malik Harris']
['Sara Kays']
['Addison Rae']
['Olivia Rodrigo']
['Musso']
['Bruno Major']
['Benson Boone']
['Scouting For Girls']
['TV Girl']
['Bishop Briggs']
['Isak Danielson']
['House Of Pain']
['Hey Violet']
['Wahlstedt']
['Walking On Cars']
['Meadowlark']
['Neon Jungle']
['Akuma']
['Liaze']
['Suki Waterhouse']
['The Fray']
['King Princess']
['The Carters']
['girl in red']
['The Local Train']
['Mac Ayres']
['Michael Schulte']
['Hearts & Colors']
['Joshua Hyslop']
['Keenan Te']
['Val

In [135]:
G.remove_nodes_from(empty)

In [136]:
np.save("../components/classification/A.npy", A)

In [137]:
nx.write_graphml(G, '../components/classification/graph.graphml')

In [138]:
df_only_selected_final = df_only_selected_final.drop(empty, axis=0).reset_index(drop=True)

In [139]:
df_only_selected_final.to_csv('../components/classification/nodes.csv')

id_map = pd.Series(df_only_selected_final.index, index=df_only_selected_final["id"]).to_dict()
name_map = pd.Series(df_only_selected_final.index, index=df_only_selected_final["name"]).to_dict()

with open("../components/classification/id_to_ind.json", "w") as file:
    json.dump(id_map, file, indent=2)

with open("../components/classification/name_to_ind.json", "w") as file:
    json.dump(name_map, file, indent=2)

ind_to_name = {v:k for (k, v) in name_map.items()}
ind_to_id = {v:k for (k, v) in id_map.items()}

with open("../components/classification/ind_to_id.json", "w") as file:
    json.dump(ind_to_id, file, indent=2)

with open("../components/classification/ind_to_name.json", "w") as file:
    json.dump(ind_to_name, file, indent=2)